In [3]:
import pandas as pd
import string
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import wordnet
import nltk

import spacy
nlp = spacy.load('en')

import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import numpy as np
from scipy.sparse.csr import csr_matrix

from sklearn.feature_extraction.text import TfidfTransformer

# ЗАДАЕМ СЛОВАРЬ ДЛЯ РАЗВОРАЧИВАНИЯ СОКРАЩЕННЫХ ФРАЗ 

In [104]:
contr = pd.read_csv("./materials/contractions.csv")
contr.head()

,short,long
0,ain t,are not
1,aren t,are not
2,cannot,can not
3,can t,can not
4,can t ve,can not have


In [105]:
contractions = {}
for key, val in zip(contr['short'],contr['long']):
    contractions[key] = val

# ЗАДАЕМ БАЗОВЫЙ СЛОВАРЬ

In [6]:
basic_vocabulary = []
with open("./materials/A1_vocab_processed.txt", "r") as voc:
    for word in voc.readlines():
        basic_vocabulary.append(word[:-1])
#basic_vocabulary = set(basic_vocabulary)
#basic_vocabulary

adjectives = []
with open("./materials/common_adj.txt", "r") as common_adj:
    for word in common_adj.readlines():
        adjectives.append(word[:-1])
        
common_uncountable = []
with open("./materials/common_unountable_manually_filtered.txt", "r") as common_unctbl:
    for word in common_unctbl.readlines():
        common_uncountable.append(word[:-1])

countries = []
with open("./materials/countries.txt", "r") as cntr:
    for word in cntr.readlines():
        countries.append(word[:-1])

names = []
with open("./materials/names.txt", "r") as names_file:
    for word in names_file.readlines():
        names.append(word[:-1])
        
print(len(basic_vocabulary), len(adjectives), len(common_uncountable), len(countries), len(countries), len(names))
final_basic_vocabulary = basic_vocabulary
final_basic_vocabulary.extend(adjectives)
final_basic_vocabulary.extend(common_uncountable)
final_basic_vocabulary.extend(countries)
final_basic_vocabulary.extend(names)
len(final_basic_vocabulary)

660 50 81 196 196 3309


4296

# ЗАДАЕМ СЛОВАРЬ ФРАЗОВЫХ ГЛАГОЛОВ

In [117]:
phrasal_list = []
with open("./materials/phrasal_verbs.txt", "r") as pv_doc:
    for pv in pv_doc:
        phrasal_list.append(pv[:-1])

# СЧИТЫВАЕМ ТЕКСТЫ

In [9]:
texts = pd.read_excel("./materials/texts.xlsx")
texts.head()

,id,Title,Text
0,1,Jimmy’s Breakfast,This is my friend Jimmy. He is from India. Jim...
1,2,A Girl from Green Valley,Green Valley was a small village near the Icy ...
2,3,Alice in Wonderland,Alice was beginning to get very tired of sitti...
3,4,Volkswagen,Volkswagen's emissions cheating will cost the ...
4,5,My Friends,My name is Sonya. I am 20 and I come from Moro...


# РАЗБИВАЕМ НА ПРЕДЛОЖЕНИЯ, ЧИСТИМ ПУНКТУАЦИЮ, ПРИВОДИМ К НИЖНЕМУ РЕГИСТРУ

In [93]:
puncuation_primary = string.punctuation + "’" +"“" + "”" + "…"
#puncuation_primary = puncuation_primary.replace(".","")#можно было сделать для сохранения аббревиатур 
#но из-за некорректной работы sentence токенайзера вызывает ошибки ассерта

In [121]:
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
#test_line = "  I can't  visit U.S. and I couldn't   to go by your daddy's car... I want to say hello to her brother.   ",
test_line = "she dreamt about being on the stage… one day director said today the king is going to visit our performance "#’'
lemmatizer = WordNetLemmatizer()
def punct_setnence_lower (line, sent_detector, debug = False, deep_debug = False):
    initial_sentences_list = sent_detector.tokenize(line.strip())
    final_sentences_list = []
    
    for sentence in initial_sentences_list:
        if(debug): print("sentence:", sentence)
        cleaned_line = ''
        #clean by word
        for word in sentence.split():
            if(debug): print(word)
            clean_word =''
            for char in word:
                if(deep_debug): print("char before cleaning:", char)
                if char not in puncuation_primary and char.isalpha():
                    clean_word += char.lower()
                    if(deep_debug): print("char after cleaning:", char.lower())
                else:
                    clean_word += ' '
                    if(deep_debug): print("char has been deleted")
            clean_word = clean_word.lstrip()
            if(debug):print("non punctuation and lower:",clean_word)       

            if (clean_word in contractions):
                if(debug):print("word before contractions parsing:",clean_word)
                clean_word = ''.join(contractions[clean_word])
                if(debug):print("word after contractions parsing:",clean_word)
            if(debug): print("finally cleaned word/s:",clean_word)    
            cleaned_line += clean_word + ' '
        
        """
        #handle abbreviations delete dots only in the end    
        for char_ind in range(len(cleaned_line) - 3, len(cleaned_line)):
            max_dot_index = len(cleaned_line)
            if cleaned_line[char_ind] == ".":
                if(char_ind < max_dot_index): max_dot_index = char_ind
                break
        cleaned_line = cleaned_line[:max_dot_index]   """ 
        cleaned_line = re.sub(' +', ' ', cleaned_line)
        cleaned_line = cleaned_line.rstrip()
        cleaned_line = cleaned_line.lstrip()
        final_sentences_list.append(cleaned_line)
    return final_sentences_list
            
clean_line = punct_setnence_lower(test_line, sent_detector,debug = False, deep_debug = False)
clean_line

['she dreamt about being on the stage one day director said today the king is going to visit our performance']

# ЗАДАЕМ ФУНКЦИЮ ДЛЯ ПОЛУЧЕНИЯ ГРАММАТИЧЕСКОЙ КАРТЫ ТЕКСТА

In [85]:
punct_setnence_lower(texts['Text'][1],sent_detector)

['green valley was a small village near the icy mountains',
 'mary lived happily there with her grandmother',
 'but one day her grandma fell ill she could not work anymore and earn enough money',
 'she asked mary to read a letter',
 'it was her granny s letter to mary s uncle',
 'he was a rich man and lived in a big city',
 'you are asking him about the job in the circus',
 'mary was very much surprised',
 'she did not want to leave her sweet home did not want to part with her beloved grandmother did not want to work in the circus',
 'but her granny said you are not a child already you are a grown up young lady',
 'you must go to the city in the city everything was different practically no trees and flowers in the streets a lot of cars and high stone buildings and much noise',
 'but people were as kind and friendly as in green valley',
 'they explained her the way to the circus and she easily found the place turned right followed a wide street turned left and here it was',
 'mary s unc

In [12]:
example_text = texts["Text"][0]
example_text_processed = punct_setnence_lower(example_text,sent_detector)
example_text_processed

['this is my friend jimmy',
 'he is from india',
 'jimmy is a vegetarian',
 'the best breakfast for him is a glass of orange juice two apples and three bananas',
 'jimmy likes watermelons very much',
 'a watermelon is green outside and red inside',
 'it is hard and soft inside',
 'it is juicy and sweet',
 'what a lovely fruit ',
 'a month ago sally received a letter',
 'it was an invitation for a wedding',
 'her sister victoria is getting married',
 'last year sally saw such a ceremony',
 'it took place on saturday',
 'there were a lot of guests people wore nice clothes met a husband and a wife near the church threw confetti ate a very tasty cake made videos and took photos',
 'in her letter victoria asked sally to be her bridesmaid',
 'it means that on the day of wedding sally must help victoria when she puts her wedding dress on and brushes her hair',
 'sally must also accompany victoria on her way to the church and carry a traditional bunch of flowers',
 'sally agreed with pleasure 

In [102]:
def get_sent_gramm_features_map(text):
    text_grammar_map = []
    for sentence in text:
        split_sent_list = sentence.split()
        grammar_map = [None] * len(split_sent_list)  
        parsed_sentence =  nlp(sentence)   
        #print(len(parsed_sentence), len(grammar_map))
        if(len(parsed_sentence) != len(split_sent_list)):
            print("ASSERTION ERROR!")
            print("parsed_sentence", len(parsed_sentence), "split_sent_list", len(split_sent_list),len(grammar_map))
            print( "split_sent_list", split_sent_list)
            for element_ind in range(len(parsed_sentence)):
                print("parsed lemma:", parsed_sentence[element_ind].lemma_, element_ind)
        assert (len(parsed_sentence) == len(grammar_map))
        for gramm_ind in range(len(split_sent_list)):
            #print(parsed_sentence[gramm_ind])
            if(parsed_sentence[gramm_ind].lemma_[0] == "-"):
                parsed_sentence[gramm_ind].lemma_ = parsed_sentence[gramm_ind].text
            grammar_map[gramm_ind] = (parsed_sentence[gramm_ind].lemma_, nlp.vocab.morphology.tag_map[parsed_sentence[gramm_ind].tag_])
        text_grammar_map.append(grammar_map)
    return text_grammar_map

grammar_map_example = get_sent_gramm_features_map(example_text_processed)
grammar_map_example[:2]

[[('this', {74: 89}),
  ('be',
   {74: 99,
    'Number': 'sing',
    'Person': 3,
    'Tense': 'pres',
    'VerbForm': 'fin'}),
  ('my', {74: 83, 'Poss': 'yes', 'PronType': 'prs'}),
  ('friend', {74: 91, 'Number': 'sing'}),
  ('jimmy', {74: 91, 'Number': 'sing'})],
 [('he', {74: 94, 'PronType': 'prs'}),
  ('be',
   {74: 99,
    'Number': 'sing',
    'Person': 3,
    'Tense': 'pres',
    'VerbForm': 'fin'}),
  ('from', {74: 84}),
  ('india', {74: 91, 'Number': 'sing'})]]

# ЛЕММАТИЗИРУЕМ ТЕКСТ

In [14]:
def get_lemm_text(text_gramm_map):
    lemm_text = []
    for sentence in text_gramm_map:
        sentence_lemm = ''
        for word in sentence:
            sentence_lemm += word[0] + ' '
        sentence_lemm = sentence_lemm[:-1]
        lemm_text.append(sentence_lemm)
        
    return lemm_text

ex_text_lemm = get_lemm_text(grammar_map_example)
ex_text_lemm[:3]

['this be my friend jimmy', 'he be from india', 'jimmy be a vegetarian']

# ВЫЧИСЛЯЕМ ТФИДФ

In [27]:
def get_tf_idf_dict(lemm_text_list, save_to_csv = False):
    vect = TfidfVectorizer(stop_words = 'english')
    tfidf_matrix = vect.fit_transform(lemm_text_list)
    df = pd.DataFrame(tfidf_matrix.toarray(), columns = vect.get_feature_names())
    #print(df.head())
    if (save_to_csv): df.to_csv("./text_0_tfidf.xlsx", sep = '\t')
    tf_idf_dict = df.to_dict()
    return tf_idf_dict
tf_idf_dict_example = get_tf_idf_dict (ex_text_lemm, save_to_csv = True)

# ЗАДАЕМ СПИСОК ВЕСОВ СЛОВ ДЛЯ ПОСЛЕДУЮЩЕГО ЗАПОЛНЕНИЯ ФИЧАМИ

In [20]:
def get_weights_empty_list(gramm_map_text):
    weights_list = []
    for sentence in gramm_map_text:
        #print(sentence)
        sentence_weights = []
        for element in sentence:
            weight = {"word" : element[0], "weight": 0}
            sentence_weights.append(weight)
        weights_list.append(sentence_weights)
    return weights_list
test_weights = get_weights_empty_list(grammar_map_example)
test_weights[:3]

[[{'weight': 0, 'word': 'this'},
  {'weight': 0, 'word': 'be'},
  {'weight': 0, 'word': 'my'},
  {'weight': 0, 'word': 'friend'},
  {'weight': 0, 'word': 'jimmy'}],
 [{'weight': 0, 'word': 'he'},
  {'weight': 0, 'word': 'be'},
  {'weight': 0, 'word': 'from'},
  {'weight': 0, 'word': 'india'}],
 [{'weight': 0, 'word': 'jimmy'},
  {'weight': 0, 'word': 'be'},
  {'weight': 0, 'word': 'a'},
  {'weight': 0, 'word': 'vegetarian'}]]

# ЗАДАЕМ ФУНКЦИЮ ДЛЯ ВЫЯВЛЕНИЯ СОЖНОЙ ГРАММАТИКИ

In [108]:
def get_difficult_grammar(text_grammar_map, weights_list, debug = False):
    for sentence_grammar_map, sentence_weights in zip(text_grammar_map,weights_list):
        for el_ind in range(len(sentence_grammar_map)):
            #print(sentence_grammar_map[el_ind])
            
            if('Aspect' in sentence_grammar_map[el_ind][1]):
                #present perfect
                if ( sentence_grammar_map[el_ind][1]['Aspect'] == 'perf' ):
                    if(sentence_grammar_map[el_ind - 2][0] == 'have' or sentence_grammar_map[el_ind - 1][0] == 'have'):
                        if(debug): print("PRESENT PERFECT")
                        if(debug): print(sentence_grammar_map[el_ind - 1])
                        if(debug): print(sentence_grammar_map[el_ind])

                        sentence_weights[el_ind]["diff_grammar"] = "pr_perf"
                        sentence_weights[el_ind - 1]["diff_grammar"] = "pr_perf"
                
                
                elif(sentence_grammar_map[el_ind][1]['Aspect'] == 'prog'):
                    #future continious check "will + be + v-ing"
                    if (sentence_grammar_map[el_ind - 1][0] == 'be' and sentence_grammar_map[el_ind - 2][0] == 'will'):
                        if(debug): print("FUTURE CONTINIOUS")                       
                        if(debug): print("prev word is", sentence_grammar_map[el_ind - 2])
                        if(debug): print("prev word is", sentence_grammar_map[el_ind - 1])
                        if(debug): print(sentence_grammar_map[el_ind])
                        sentence_weights[el_ind]["diff_grammar"] = "fut_cont"
                        sentence_weights[el_ind - 1]["diff_grammar"] = "fut_cont"
                        sentence_weights[el_ind - 2]["diff_grammar"] = "fut_cont"
                   #past continious was/were + v-ing"     
                    elif (sentence_grammar_map[el_ind - 1][0] == 'be' and 'Tense' in sentence_grammar_map[el_ind -1][1]):
                        if(sentence_grammar_map[el_ind -1][1]['Tense'] == 'past'):
                            if(debug): print("PAST CONTINIOUS")
                            if(debug): print(sentence_grammar_map[el_ind])
                            if(debug): print("prev word is", sentence_grammar_map[el_ind - 1])
                            sentence_weights[el_ind]["diff_grammar"] = "past_cont"
                            sentence_weights[el_ind - 1]["diff_grammar"] = "past_cont"
                        
                
    return weights_list

test_weights = get_difficult_grammar(grammar_map_example,test_weights, debug = True)
test_weights[:3]

[[{'weight': 0.05, 'word': 'this'},
  {'weight': 0.05, 'word': 'be'},
  {'weight': 0.05, 'word': 'my'},
  {'weight': 0.774119582428778, 'word': 'friend'},
  {'weight': 0.6330393922184421, 'word': 'jimmy'}],
 [{'weight': 0.05, 'word': 'he'},
  {'weight': 0.05, 'word': 'be'},
  {'weight': 0.05, 'word': 'from'},
  {'weight': 1.0, 'word': 'india'}],
 [{'weight': 0.6330393922184421, 'word': 'jimmy'},
  {'weight': 0.05, 'word': 'be'},
  {'weight': 0.05, 'word': 'a'},
  {'weight': 0.774119582428778, 'word': 'vegetarian'}]]

# ЗАДАЕМ ФНУКЦИЮ ДЛЯ ПОИСКА ФРАЗОВЫХ ГЛАГОЛОВ

In [22]:
def get_phrasal_verbs(text_grammar_map, weights_list, debug = False):
    for sentence_grammar_map, sentence_weights in zip(text_grammar_map,weights_list):
        for el_ind in range(1, len(sentence_grammar_map)):  
            for searh_ind in range(1, 3):
                try_phrase = sentence_grammar_map[el_ind - searh_ind][0] + ' ' + sentence_grammar_map[el_ind][0]
                #print(try_phrase)
                if(try_phrase in phrasal_list):
                    if(debug): print("Phrasal Verb found:", try_phrase)
                    sentence_weights[el_ind]["phrasal_verb"] = "yes"
                    sentence_weights[el_ind - searh_ind]["phrasal_verb"] = "yes"
    return weights_list
        #print("\n")
test_weights = get_phrasal_verbs(grammar_map_example,test_weights,debug = True)
test_weights[-2:]

Phrasal Verb found: agree with
Phrasal Verb found: add up
Phrasal Verb found: give up


[[{'weight': 0, 'word': 'she'},
  {'weight': 0, 'word': 'be'},
  {'weight': 0, 'word': 'dream'},
  {'weight': 0, 'word': 'about'},
  {'weight': 0, 'word': 'a'},
  {'weight': 0, 'word': 'slice'},
  {'weight': 0, 'word': 'of'},
  {'weight': 0, 'word': 'wedding'},
  {'weight': 0, 'word': 'cake'},
  {'weight': 0, 'word': 'and'},
  {'weight': 0, 'word': 'do'},
  {'weight': 0, 'word': 'not'},
  {'weight': 0, 'word': 'want'},
  {'weight': 0, 'word': 'to'},
  {'phrasal_verb': 'yes', 'weight': 0, 'word': 'give'},
  {'phrasal_verb': 'yes', 'weight': 0, 'word': 'up'}],
 [{'weight': 0, 'word': 'dieting'},
  {'weight': 0, 'word': 'be'},
  {'weight': 0, 'word': 'not'},
  {'weight': 0, 'word': 'easy'}]]

# ПРИСВАИВАЕМ ВЕСА ТФИДФ

In [30]:
def assign_tf_idf(text_grammar_map, weights_list, tf_idf_dict):
    assert (len(text_grammar_map) == len(weights_list))
    for sentence_ind in range(len(text_grammar_map)):
        for el_ind in range(len(text_grammar_map[sentence_ind])):
            lemma = weights_list[sentence_ind][el_ind]["word"]
            
            if (lemma in tf_idf_dict):
                weights_list[sentence_ind][el_ind]["weight"] = tf_idf_dict[lemma][sentence_ind]
                #print(lemma, tf_idf_dict[lemma][sentence_ind])
            else:
                weights_list[sentence_ind][el_ind]["weight"] = 0.05
                #print(lemma, "not found")
                
    return weights_list
test_weights = assign_tf_idf(grammar_map_example, test_weights, tf_idf_dict_example)
test_weights[:10]        

[[{'weight': 0.05, 'word': 'this'},
  {'weight': 0.05, 'word': 'be'},
  {'weight': 0.05, 'word': 'my'},
  {'weight': 0.774119582428778, 'word': 'friend'},
  {'weight': 0.6330393922184421, 'word': 'jimmy'}],
 [{'weight': 0.05, 'word': 'he'},
  {'weight': 0.05, 'word': 'be'},
  {'weight': 0.05, 'word': 'from'},
  {'weight': 1.0, 'word': 'india'}],
 [{'weight': 0.6330393922184421, 'word': 'jimmy'},
  {'weight': 0.05, 'word': 'be'},
  {'weight': 0.05, 'word': 'a'},
  {'weight': 0.774119582428778, 'word': 'vegetarian'}],
 [{'weight': 0.05, 'word': 'the'},
  {'weight': 0.3779644730092272, 'word': 'good'},
  {'weight': 0.3779644730092272, 'word': 'breakfast'},
  {'weight': 0.05, 'word': 'for'},
  {'weight': 0.05, 'word': 'him'},
  {'weight': 0.05, 'word': 'be'},
  {'weight': 0.05, 'word': 'a'},
  {'weight': 0.3779644730092272, 'word': 'glass'},
  {'weight': 0.05, 'word': 'of'},
  {'weight': 0.3779644730092272, 'word': 'orange'},
  {'weight': 0.3779644730092272, 'word': 'juice'},
  {'weight': 

# ДЕЛИМ НА СЛОЖНЫЕ И НЕ СЛОЖНЫЕ В СООТВЕТСВИИ С ПОЛУЧЕННЫМИ ФИЧАМИ

In [31]:
def split_into_groups(text_weights):
    difficult_vocabulary = []
    easy_vocabulary = []
    for sentence_weights in text_weights:
        for word_weight in sentence_weights:
            #print(word_weight)
            if('diff_grammar' in word_weight or 'phrasal_verb' in word_weight or word_weight['word'] not in basic_vocabulary):
                difficult_vocabulary.append(word_weight)
            else:
                easy_vocabulary.append(word_weight)
    return easy_vocabulary, difficult_vocabulary
                
easy, difficult = split_into_groups(test_weights)

In [32]:
len(easy), len(difficult)

(252, 62)

In [33]:
difficult

[{'weight': 0.774119582428778, 'word': 'vegetarian'},
 {'weight': 0.5688125307495037, 'word': 'watermelon'},
 {'weight': 0.4261771001650547, 'word': 'watermelon'},
 {'weight': 0.5978117297300675, 'word': 'soft'},
 {'weight': 0.774119582428778, 'word': 'juicy'},
 {'weight': 0.7457391260354046, 'word': 'lovely'},
 {'weight': 0.4878326589920418, 'word': 'receive'},
 {'weight': 0.774119582428778, 'word': 'invitation'},
 {'weight': 0.6330393922184421, 'word': 'wedding'},
 {'weight': 0.6230307846633135, 'word': 'marry'},
 {'weight': 0.05, 'word': 'such'},
 {'weight': 0.6020488713383773, 'word': 'ceremony'},
 {'weight': 0.23674868832917667, 'word': 'guest'},
 {'weight': 0.23674868832917667, 'word': 'clothe'},
 {'weight': 0.211509604231399, 'word': 'church'},
 {'weight': 0.23674868832917667, 'word': 'throw'},
 {'weight': 0.23674868832917667, 'word': 'confetti'},
 {'weight': 0.23674868832917667, 'word': 'tasty'},
 {'weight': 0.23674868832917667, 'word': 'video'},
 {'weight': 0.5144649816247601,

In [126]:
def calculate_level(easy_words_weights, difficults_words_weights, debug = False):
    tf_idf_easy = 0
    for easy_w_w in easy_words_weights:
        tf_idf_easy += easy_w_w['weight']
        
    tf_idf_diff = 0
    for diff_w_w in difficults_words_weights:
        tf_idf_diff += diff_w_w['weight']
        
    if(debug):print("easy words:", len(easy_words_weights), "difficult words:", len(difficults_words_weights))
    if(debug):print("easy tfidf sum:",tf_idf_easy, "easy tfidf sum:",tf_idf_diff)
    overall_weights = tf_idf_easy + tf_idf_diff
    if(debug):print("relative tfidf", round(tf_idf_easy/overall_weights, 2), round(tf_idf_diff/overall_weights, 2))
        
    if(round(tf_idf_diff/overall_weights, 2) >= 0.3):
        print("text is very diffcult, diff realtive weight is more then 0.3")
        easy_text = False
    if(round(tf_idf_diff/overall_weights, 2) > 0.2 and round(tf_idf_diff/overall_weights, 2) < 0.3):
        print("text is moderate")
        easy_text = False
    if(round(tf_idf_diff/overall_weights, 2) <= 0.2):
        print("text is quite easy")
        easy_text = True
        
    
        
    return easy_text
calculate_level(easy, difficult,debug = True)

easy words: 252 difficult words: 62
easy tfidf sum: 47.279379344440464 easy tfidf sum: 26.094957600305506
relative tfidf 0.64 0.36
text is very diffcult, diff realtive weight is more then 0.3


False

In [127]:
def calculate_text_level(raw_text, print_debug_message = False, show_calucated_weights = False):
    processed_text = punct_setnence_lower(raw_text,sent_detector)
    
    grammar_map = get_sent_gramm_features_map(processed_text)
    
    text_lemm = get_lemm_text(grammar_map)
    
    tfidf_dict = get_tf_idf_dict(text_lemm)
    
    weights = get_weights_empty_list(grammar_map)
    
    weights = get_difficult_grammar(grammar_map,weights, debug = print_debug_message)
    weights = get_phrasal_verbs(grammar_map,weights, debug = print_debug_message)
    weights = assign_tf_idf(grammar_map, weights, tfidf_dict)
    
    if(show_calucated_weights): 
        print("WEIGHTS HAVE BEEN CALCULATED AS FOLLOWS (sentence by sentence)")
        for sent_weights in weights:
            for word_weight in sent_weights:
                print(word_weight)
            print("\n")
    easy, difficult = split_into_groups(weights)
    
    easy_text = calculate_level(easy, difficult,  debug = print_debug_message)
    
    return easy_text

# ВЫВОДИМ РЕЗУЛЬТАТЫ ПО ВСЕМ ТЕКСТАМ

In [130]:
easy_texts_indexes = []
for text_ind in range(len(texts['Text'])):
    print("=====================================")
    print(texts["Title"][text_ind], "level is calculated")
    print("=====================================")
    easy_text = calculate_text_level(texts['Text'][text_ind])
    if(easy_text):
        easy_texts_indexes.append(text_ind) 
    print("\n")

Jimmy’s Breakfast level is calculated
text is very diffcult, diff realtive weight is more then 0.3


A Girl from Green Valley level is calculated
text is very diffcult, diff realtive weight is more then 0.3


Alice in Wonderland level is calculated
text is moderate


Volkswagen level is calculated
text is very diffcult, diff realtive weight is more then 0.3


My Friends level is calculated
text is moderate


Good Advice From A Doctor level is calculated
text is moderate


About The Weather level is calculated
text is quite easy


A Lucky Puppy level is calculated
text is moderate


Anne of Green Gables level is calculated
text is very diffcult, diff realtive weight is more then 0.3


We Like Sports! level is calculated
text is quite easy


James Bond 1 level is calculated
text is very diffcult, diff realtive weight is more then 0.3


Charlotte’s Web level is calculated
text is very diffcult, diff realtive weight is more then 0.3




In [131]:
easy_texts_indexes

[6, 9]

In [116]:
for text_ind in range(len(texts['Text'])):
    print("=====================================")
    print(texts["Title"][text_ind], "level is calculated")
    print("=====================================")
    calculate_text_level(texts['Text'][text_ind],print_debug_message = True, show_calucated_weights = True)
    print("\n")    

Jimmy’s Breakfast level is calculated
Phrasal Verb found: agree with
Phrasal Verb found: add up
Phrasal Verb found: give up
WEIGHTS HAVE BEEN CALCULATED AS FOLLOWS (sentence by sentence)
{'word': 'this', 'weight': 0.05}
{'word': 'be', 'weight': 0.05}
{'word': 'my', 'weight': 0.05}
{'word': 'friend', 'weight': 0.774119582428778}
{'word': 'jimmy', 'weight': 0.6330393922184421}


{'word': 'he', 'weight': 0.05}
{'word': 'be', 'weight': 0.05}
{'word': 'from', 'weight': 0.05}
{'word': 'india', 'weight': 1.0}


{'word': 'jimmy', 'weight': 0.6330393922184421}
{'word': 'be', 'weight': 0.05}
{'word': 'a', 'weight': 0.05}
{'word': 'vegetarian', 'weight': 0.774119582428778}


{'word': 'the', 'weight': 0.05}
{'word': 'good', 'weight': 0.3779644730092272}
{'word': 'breakfast', 'weight': 0.3779644730092272}
{'word': 'for', 'weight': 0.05}
{'word': 'him', 'weight': 0.05}
{'word': 'be', 'weight': 0.05}
{'word': 'a', 'weight': 0.05}
{'word': 'glass', 'weight': 0.3779644730092272}
{'word': 'of', 'weight'

{'word': 'to', 'weight': 0.05}
{'word': 'leave', 'weight': 0.23422099531941273}
{'word': 'her', 'weight': 0.05}
{'word': 'sweet', 'weight': 0.26269408862536386}
{'word': 'home', 'weight': 0.23422099531941273}
{'word': 'do', 'weight': 0.05}
{'word': 'not', 'weight': 0.05}
{'word': 'want', 'weight': 0.7880822658760916}
{'word': 'to', 'weight': 0.05}
{'word': 'part', 'weight': 0.05}
{'word': 'with', 'weight': 0.05}
{'word': 'her', 'weight': 0.05}
{'word': 'beloved', 'weight': 0.26269408862536386}
{'word': 'grandmother', 'weight': 0.23422099531941273}
{'word': 'do', 'weight': 0.05}
{'word': 'not', 'weight': 0.05}
{'word': 'want', 'weight': 0.7880822658760916}
{'word': 'to', 'weight': 0.05}
{'word': 'work', 'weight': 0.21401901411913682}
{'word': 'in', 'weight': 0.05}
{'word': 'the', 'weight': 0.05}
{'word': 'circus', 'weight': 0.1747209531350138}


{'word': 'but', 'weight': 0.05}
{'word': 'her', 'weight': 0.05}
{'word': 'granny', 'weight': 0.33004178111619376}
{'word': 'say', 'weight': 0.3

{'word': 'own', 'weight': 0.05}
{'word': 'mind', 'weight': 0.193706381806703}
{'word': 'as', 'weight': 0.05}
{'word': 'well', 'weight': 0.05}
{'word': 'as', 'weight': 0.05}
{'word': 'she', 'weight': 0.05}
{'word': 'could', 'weight': 0.05}
{'word': 'for', 'weight': 0.05}
{'word': 'the', 'weight': 0.05}
{'word': 'hot', 'weight': 0.193706381806703}
{'word': 'day', 'weight': 0.193706381806703}
{'word': 'make', 'weight': 0.387412763613406}
{'word': 'her', 'weight': 0.05}
{'word': 'feel', 'weight': 0.193706381806703}
{'word': 'very', 'weight': 0.05}
{'word': 'sleepy', 'weight': 0.193706381806703}
{'word': 'and', 'weight': 0.05}
{'word': 'stupid', 'weight': 0.193706381806703}
{'word': 'whether', 'weight': 0.05}
{'word': 'the', 'weight': 0.05}
{'word': 'pleasure', 'weight': 0.193706381806703}
{'word': 'of', 'weight': 0.05}
{'word': 'make', 'weight': 0.387412763613406}
{'word': 'a', 'weight': 0.05}
{'word': 'daisy', 'weight': 0.387412763613406}
{'word': 'chain', 'weight': 0.193706381806703}
{'w

{'word': 'to', 'weight': 0.05}
{'word': 'listen', 'weight': 0.25109483081077144}
{'word': 'to', 'weight': 0.05}
{'word': 'loud', 'weight': 0.25109483081077144}
{'word': 'music', 'weight': 0.25109483081077144}
{'word': 'and', 'weight': 0.05}
{'word': 'sleep', 'weight': 0.25109483081077144}
{'word': 'more', 'weight': 0.05}


{'word': 'when', 'weight': 0.05}
{'word': 'yuko', 'weight': 0.46803237682201176}
{'word': 'have', 'weight': 0.05}
{'word': 'a', 'weight': 0.05}
{'word': 'toothache', 'weight': 0.46803237682201176}
{'word': 'i', 'weight': 0.05}
{'word': 'say', 'weight': 0.46803237682201176}
{'word': 'do', 'weight': 0.05}
{'word': 'not', 'weight': 0.05}
{'word': 'eat', 'weight': 0.3518277660807251}
{'word': 'sweet', 'weight': 0.46803237682201176}


{'word': 'eat', 'weight': 0.2933825421139075}
{'word': 'more', 'weight': 0.05}
{'word': 'vegetable', 'weight': 0.3902833765318864}
{'word': 'and', 'weight': 0.05}
{'word': 'fruit', 'weight': 0.3902833765318864}
{'word': 'roby', 'weight': 0.3

{'word': 'not', 'weight': 0.05}
{'word': 'drive', 'weight': 0.35355339059327373}
{'word': 'fast', 'weight': 0.35355339059327373}
{'word': 'enough', 'weight': 0.05}
{'word': 'to', 'weight': 0.05}
{'word': 'be', 'weight': 0.05}
{'word': 'go', 'weight': 0.05}
{'word': 'for', 'weight': 0.05}
{'word': 'a', 'weight': 0.05}
{'word': 'doctor', 'weight': 0.35355339059327373}


{'word': 'yet', 'weight': 0.05}
{'word': 'something', 'weight': 0.05}
{'word': 'must', 'weight': 0.05}
{'word': 'have', 'weight': 0.05, 'diff_grammar': 'pr_perf'}
{'word': 'happen', 'weight': 0.7071067811865475, 'diff_grammar': 'pr_perf'}
{'word': 'since', 'weight': 0.05}
{'word': 'last', 'weight': 0.05}
{'word': 'night', 'weight': 0.7071067811865475}


{'word': 'I', 'weight': 0.05}
{'word': 'be', 'weight': 0.05}
{'word': 'puzzle', 'weight': 0.3222883300855695}
{'word': 'and', 'weight': 0.05}
{'word': 'i', 'weight': 0.05}
{'word': 'will', 'weight': 0.05}
{'word': 'not', 'weight': 0.05}
{'word': 'know', 'weight': 0.6445766

easy tfidf sum: 13.611327050628047 easy tfidf sum: 15.698806025880634
relative tfidf 0.46 0.54
text is very diffcult, diff realtive weight is more then 0.3


Charlotte’s Web level is calculated
PAST CONTINIOUS
('set', {74: 99, 'VerbForm': 'part', 'Tense': 'pres', 'Aspect': 'prog'})
prev word is ('be', {74: 99, 'VerbForm': 'fin', 'Tense': 'past'})
PRESENT PERFECT
('have', {74: 99, 'VerbForm': 'fin', 'Tense': 'pres', 'Number': 'sing', 'Person': 3})
('decide', {74: 99, 'VerbForm': 'part', 'Tense': 'past', 'Aspect': 'perf'})
WEIGHTS HAVE BEEN CALCULATED AS FOLLOWS (sentence by sentence)
{'word': 'where', 'weight': 0.05}
{'word': 'be', 'weight': 0.05}
{'word': 'papa', 'weight': 0.751746077547996}
{'word': 'go', 'weight': 0.05}
{'word': 'with', 'weight': 0.05}
{'word': 'that', 'weight': 0.05}
{'word': 'ax', 'weight': 0.6594526782804071}


{'word': 'say', 'weight': 0.36804737525465214}
{'word': 'fern', 'weight': 0.3108586726281349}
{'word': 'to', 'weight': 0.05}
{'word': 'her', 'weight': 0.05